In [1]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd gdrive/My\ Drive/

/content/gdrive/My Drive


In [4]:
!ls -l | grep "bg"

-rw------- 1 root root   49453404 May 10 10:14 bg.zip
-rw------- 1 root root 2289574765 May 10 10:01 dataset_fg_bg_final
-rw------- 1 root root  716783927 May 10 10:01 dataset_fg_bg_Mask_final
-rw------- 1 root root 2114732054 May 10 15:52 depth_fg_bg (1).zip
-rw------- 1 root root 1480413706 May 10 08:45 depth_fg_bg (2).zip
-rw------- 1 root root 2114732054 May 10 15:52 depth_fg_bg.zip


In [5]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from PIL import Image
import torchvision.transforms as T
import os
import torch
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Let's see if the dataset works

In [0]:
depth_fg_bg_zip =  ZipFile('depth_fg_bg.zip', 'r')

In [0]:
all_files = [info.filename for info in depth_fg_bg_zip.infolist() if not info.is_dir()]

In [8]:
len(all_files)

269273

In [0]:
!ls -l | grep "bg"

-rw------- 1 root root   49453404 May 10 10:14 bg.zip
-rw------- 1 root root 2289574765 May 10 10:01 dataset_fg_bg_final
-rw------- 1 root root  716783927 May 10 10:01 dataset_fg_bg_Mask_final
-r-------- 1 root root 1585928873 May 10 10:01 depth_fg_bg (1).zip
-rw------- 1 root root 1480413706 May 10 08:45 depth_fg_bg (2).zip
-r-------- 1 root root 1607107207 May 10 10:14 depth_fg_bg.zip


In [0]:
fg_bg_zip =  ZipFile('dataset_fg_bg_Mask_final', 'r')

In [0]:
all_files = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]

In [8]:
len(all_files)

400000

In [0]:
class DenseDepth(Dataset):
    '''
    DenseDepth Dataset

    Input is fg_bg image AND bg image
    Target is fg_bg_mask AND depth_fg_bg image
    '''
    def __init__(self, root, data='fg_bg', train=True, transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform

        # open the respective data and target
        bg_zip = ZipFile(os.path.join(self.root,'bg.zip'), 'r') # bg
        fg_bg_zip =  ZipFile(os.path.join(self.root,'dataset_fg_bg_final'), 'r') # fg_bg
        fg_bg_mask_zip = ZipFile(os.path.join(self.root,'dataset_fg_bg_Mask_final'), 'r') # fg_bg_mask
        depth_fg_bg_zip =  ZipFile(os.path.join(self.root,'depth_fg_bg.zip'), 'r') # target

        bg_paths = [info.filename for info in bg_zip.infolist() if not info.is_dir()]
        fg_bg_paths = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]
        fg_bg_mask_paths = [info.filename for info in fg_bg_mask_zip.infolist() if not info.is_dir()]
        depth_fg_bg_paths = [info.filename for info in depth_fg_bg_zip.infolist() if not info.is_dir()]


        assert(len(bg_paths) == 100)
        assert(len(fg_bg_paths) == 400000)
        assert(len(fg_bg_mask_paths) == 400000)
        assert(len(depth_fg_bg_paths) == 269273)

        print(f'found {len(bg_paths)} bg images, {len(fg_bg_paths)} fg_bg images, {len(fg_bg_mask_paths)} fg_bg_mask images, {len(depth_fg_bg_paths)} depth_fg_bg images')


        if data == 'fg_bg':
            self.image_zip = fg_bg_zip
            self.image_paths = fg_bg_paths
        elif data == 'bg':
            self.image_zip = bg_zip
            self.image_paths = bg_paths
        elif data == 'fg_bg_mask':
            self.image_zip = fg_bg_mask_zip
            self.image_paths = fg_bg_mask_paths
        elif data == 'depth_fg_bg':
            self.image_zip = depth_fg_bg_zip
            self.image_paths = depth_fg_bg_paths
        else:
            raise f'{data} is not a valid option'

        self.targets = []

    def __getitem__(self, index):
        imgdata = self.image_zip.read(self.image_paths[index])
        img = Image.open(io.BytesIO(imgdata))
        img = img.convert("RGB")
        img = np.array(img)

        if self.transform is not None:
            img = self.transform(img)
        
        return img

    def __len__(self):
        return len(self.image_paths)

In [0]:
del depth_dataset, depth_dataloader

In [29]:
depth_dataset = DenseDepth(root='./', transform=T.Compose([T.ToTensor()]))

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 269273 depth_fg_bg images


In [0]:
depth_dataloader = DataLoader(depth_dataset, batch_size=4, shuffle=True, num_workers=1)

In [0]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for images in tqdm(dataloader):
        for i in range(3):
            mean[i] += images[:, i, :, :].mean()
            std[i] += images[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

## Compute Mean and Std dev. for BG images

In [13]:
bg_dataset = DenseDepth(root='./', data='bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images
==> Computing mean and std..


In [14]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.445187121629715', '0.453919261693954', '0.442775547504425'],
 ['0.209829762578011', '0.204434663057327', '0.218459382653236'])

## Compute Mean and Std dev. for fg_bg images

In [15]:
fg_bg_dataset = DenseDepth(root='./', data='fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images
==> Computing mean and std..


In [16]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.430746674537659', '0.430641651153564', '0.428456991910934'],
 ['0.233161732554436', '0.229639455676079', '0.240961566567421'])

## Compute Mean and Std dev. for fg_bg_mask

In [17]:
fg_bg_mask_dataset = DenseDepth(root='./', data='fg_bg_mask', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_mask_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images
==> Computing mean and std..


In [18]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.246706947684288', '0.246706947684288', '0.246706947684288'],
 ['0.419592112302780', '0.419592112302780', '0.419592112302780'])

## Compute Mean and Std dev. for depth_fg_bg

In [32]:
depth_fg_bg_dataset = DenseDepth(root='./', data='depth_fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(depth_fg_bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 269273 depth_fg_bg images
==> Computing mean and std..


In [33]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.185273632407188', '0.185273632407188', '0.185273632407188'],
 ['0.056562200188637', '0.056562200188637', '0.056562200188637'])